 Andrew J Markland, ajm259@uakron.edu\
 The University of Akron, CEPS, School of Computer Science\
 Applied Machine Learning CPSC-436-010\
 Dr. Zhong-Hui Duan\

# Project 1, kNN
 -Project Scope:\
     Apply the machine learning classification algorithm "k nearest neighbors"(kNN) to correctly classify a dataset of people based on their risk for MI, Myocardial Infarction.  

In [7]:
#imports 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from pandas.plotting import scatter_matrix

In [11]:
people_df = pd.read_csv("NHANES_data_train.csv")
#count of people with MI's and total without MI's
MICount = (people_df["MI"] == 1).sum()
print(MICount)


145


      ParticipantID  Income  Sex  Age  Race  Edu  Diastolic    Systolic  \
0                 1    4.56    1   70     3  NaN  69.000000  124.333333   
1                 2    5.00    2   49     3  5.0  64.000000  102.666667   
2                 3    1.59    1   54     4  NaN  96.000000  160.000000   
3                 4    2.37    1   73     3  NaN  58.000000  134.666667   
4                 5    5.00    1   42     3  NaN  82.333333  116.000000   
...             ...     ...  ...  ...   ...  ...        ...         ...   
4074           4075    1.32    2   68     2  1.0  46.666667  116.000000   
4075           4076    0.68    1   51     3  NaN  59.333333  105.666667   
4076           4077    1.02    2   54     3  3.0  62.666667  107.333333   
4077           4078     NaN    2   48     4  4.0  74.000000  126.666667   
4078           4079    1.57    2   61     4  NaN  76.666667  115.333333   

          Pulse   BMI  HDL  Trig  LDL  TCHOL  kidneys_eGFR  Diabetes  \
0     61.333333  25.6   46 